In [1]:
import csv
import numpy as np
import dendropy
import geopandas as gpd
from collections import defaultdict
from collections import Counter

In [2]:
location_dict = defaultdict(dict)
with open("../genomic_data/study_dataset.csv") as f:
    data = csv.DictReader(f)
    for l in data:
        location_dict[l['fasta_name']] = {"state":l['state'], "county":l['county'], "site":l['site']}
        
        

In [3]:
north_east = ["Connecticut", "Massachusetts", "New_Hampshire", "Vermont", "Rhode_Island", "New_York"]


In [4]:
def find_change_individual(nde, sample_dict, location, annotation):
    if nde.taxon:
        if nde.annotations[annotation].value == location:
            sample_dict["sample"] = (nde.annotations['height'].value, nde.annotations['height_95%_HPD'].value)
            if nde.parent_node.annotations[annotation].value != location:
                sample_dict["change_node"] = nde.parent_node
                sample_dict["change_time"] = (nde.parent_node.annotations['height'].value, nde.parent_node.annotations['height_95%_HPD'].value)
                return sample_dict
            else:
                find_change_individual(nde.parent_node, sample_dict, location)
                return sample_dict
            
        else:
            return
    else:
        if nde.parent_node.annotations[annotation].value != location:
            sample_dict["change_node"] = nde.parent_node
            sample_dict["change_time"] = (nde.parent_node.annotations['height'].value, nde.parent_node.annotations['height_95%_HPD'].value)
            return sample_dict
        else:
            find_change_individual(nde.parent_node, sample_dict, location)
            return sample_dict
        
        
def find_change_north_east(nde, sample_dict):
    if nde.taxon:
        if nde.annotations['state'].value in north_east:
            sample_dict["sample"] = (nde.annotations['height'].value, nde.annotations['height_95%_HPD'].value)
            if nde.parent_node.annotations['state'].value not in north_east:
                sample_dict["change_node"] = nde.parent_node
                sample_dict["change_time"] = (nde.parent_node.annotations['height'].value, nde.parent_node.annotations['height_95%_HPD'].value)
                return sample_dict
            else:
                find_change_north_east(nde.parent_node, sample_dict)
                return sample_dict
            
        else:
            return
    else:
        if nde.parent_node.annotations['state'].value not in north_east:
            sample_dict["change_node"] = nde.parent_node
            sample_dict["change_time"] = (nde.parent_node.annotations['height'].value, nde.parent_node.annotations['height_95%_HPD'].value)
            return sample_dict
        else:
            find_change_north_east(nde.parent_node, sample_dict)
            return sample_dict
        
        
def find_new_change(nde, sample_dict):
    
    if nde.taxon:
        if nde.annotations['new_transition'].value == "North_East":
            sample_dict["sample"] = (nde.annotations['height'].value, nde.annotations['height_95%_HPD'].value)
            if nde.parent_node.annotations['new_transition'].value != "North_East":
                sample_dict["change_node"] = nde.parent_node
                sample_dict["change_time"] = (nde.parent_node.annotations['height'].value, nde.parent_node.annotations['height_95%_HPD'].value)
                return sample_dict
            else:
                sample_dict = find_new_change(nde.parent_node, sample_dict)
                return sample_dict
            
        else:
            return
    else:
        if nde.annotations['new_transition'].value != "North_East":
            sample_dict["change_node"] = nde.parent_node
            sample_dict["change_time"] = (nde.parent_node.annotations['height'].value, nde.parent_node.annotations['height_95%_HPD'].value)
            return sample_dict
        else:
            find_change_north_east(nde.parent_node, sample_dict)
            return sample_dict
        
def get_clusters(tree, change_function, state=None):
    
    path_dict = defaultdict(dict)
    for leaf in tree.leaf_iter():
        if state:
            sample_dict = change_function(leaf, {}, state)
        else:
            sample_dict = change_function(leaf, {})
        path_dict[leaf.taxon.label] = sample_dict
                            
    clusters = defaultdict(list)
    for tip, change in path_dict.items():
        if change:
            clusters[change['change_node']].append(tip)

    return clusters, path_dict

In [6]:
#tree that is output from the chopping up introductions script - ie in "breaking up DTA introductions"
tree = dendropy.Tree.get(path="new_introductions.tree", schema="nexus", preserve_underscores=True)

In [7]:
ne_clusters, ne_path_dict = get_clusters(tree, find_new_change)
print(len(ne_clusters))

65


/var/folders/pd/bjtp4xt916vfpszlps7rr5_c0000gn/T/ipykernel_35938/2990361805.py:76: CriticalDeprecationWarning: Deprecated since DendroPy 4: 'leaf_iter()' will no longer be supported in future releases; use 'leaf_node_iter()' instead
  for leaf in tree.leaf_iter():


In [8]:
state_counts = defaultdict(dict)
county_counts = defaultdict(dict)
site_counts = defaultdict(dict)
singletons = []
for node, cluster in ne_clusters.items():
    if len(cluster) > 1:
        state_lst = []
        county_lst = []
        site_lst = []
        for tip in cluster:
            if location_dict[tip]['state'] != "USA":
                state_lst.append(location_dict[tip]["state"])
                county_lst.append(location_dict[tip]["county"])
                site_lst.append(location_dict[tip]['site'])
        state_counts[node] = Counter(state_lst)
        county_counts[node] = Counter(county_lst)
        site_counts[node] = Counter(site_lst)
    else:
        singletons.append(cluster[0])

In [9]:
multi_state_outbreaks = []
multi_county_outbreaks = []
single_county_outbreaks = []
for k,v in state_counts.items():
    if len(v) > 1:
        multi_state_outbreaks.append(k)
    else:
        if len(county_counts[k]) > 1: #only those without counties have only 1
            multi_county_outbreaks.append(k)
        else:
            single_county_outbreaks.append(k)

In [10]:
def path_to_tip(nde, lst):
    
    for child in nde.child_node_iter():
        if child.annotations['state'].value in north_east:
            if not child.taxon:
                lst.append(child)
                lst = path_to_tip(child, lst)
            else:
                lst.append(child.taxon.label)
        else:
            continue
            
    return lst

In [12]:
multi_states = defaultdict(dict)
for nde in multi_state_outbreaks:
    loc_dict = defaultdict(list)
    mrcas = {}
    paths = {}
    polyphyletic = {}
    spillover = {}
    poss_monos = []
    monos = set()
    
    for tip in ne_clusters[nde]:
        loc_dict[location_dict[tip]['state']].append(tip)
    for state, tips in loc_dict.items():
        mrcas[state] = tree.mrca(taxon_labels=tips) 
    for mrca in mrcas.values():
        paths[mrca] = path_to_tip(mrca, [])
            
    for state, mrca in mrcas.items():
        for state2, mrca2 in mrcas.items():
            if state != state2:
                if mrca == mrca2:
                    polyphyletic[state] = state2
                else:
                    poss_monos.append((state,state2))
                                        
    for state_pairs in poss_monos:
        mrca1 = mrcas[state_pairs[0]]
        mrca2 = mrcas[state_pairs[1]]
        
        if float(mrca1.annotations['height'].value) > float(mrca2.annotations['height'].value):
            if mrca2 not in paths[mrca1]:
                monos.add(state_pairs[0])
                monos.add(state_pairs[1])
            else:
                monos.add(state_pairs[1])
                spillover[state_pairs[0]] = state_pairs[1]
        else:
            if mrca1 not in paths[mrca2]:
                monos.add(state_pairs[1])
                monos.add(state_pairs[0])
            else:
                monos.add(state_pairs[0])
                spillover[state_pairs[1]] = state_pairs[0]
                
    
    multi_states[nde]["polys"] = polyphyletic
    multi_states[nde]["monos"] = monos
    multi_states[nde]["spillover"] = spillover

In [13]:
for k,v in multi_states.items():
    print(v['monos'])

{'New_York', 'Massachusetts'}
{'Connecticut', 'New_York', 'Massachusetts'}
{'Connecticut', 'New_York', 'Massachusetts'}
{'Connecticut', 'New_York'}
set()
{'Connecticut', 'New_York'}
{'New_York', 'Massachusetts'}
{'Connecticut', 'Massachusetts'}
{'New_Hampshire'}
{'New_York', 'Massachusetts'}
{'New_York', 'Vermont', 'Massachusetts'}
{'New_York', 'Massachusetts'}


In [14]:
for k,v in multi_states.items():
    print(v['spillover'])

{}
{'Massachusetts': 'Connecticut'}
{'Connecticut': 'Massachusetts'}
{}
{}
{}
{}
{}
{'Massachusetts': 'New_Hampshire'}
{}
{}
{}


In [15]:
for k,v in multi_states.items():
    print(v['polys'])

{}
{}
{}
{}
{'New_York': 'Massachusetts', 'Massachusetts': 'New_York'}
{}
{}
{}
{}
{}
{}
{}


In [157]:
multi_counties = defaultdict(dict)
for nde in multi_county_outbreaks:
    loc_dict = defaultdict(list)
    mrcas = {}
    paths = {}
    polyphyletic = {}
    spillover = {}
    poss_monos = set()
    monos = set()
    
    for tip in clusters[nde]:
        loc_dict[location_dict[tip]['county']].append(tip)
    for county, tips in loc_dict.items():
        if county != "":
            mrcas[county] = tree.mrca(taxon_labels=tips) 
    for mrca in mrcas.values():
        paths[mrca] = path_to_tip(mrca, [])
            
    for county, mrca in mrcas.items():
        for county2, mrca2 in mrcas.items():
            if county != county2:
                if mrca == mrca2:
                    polyphyletic[county] = county2
                else:
                    poss_monos.add(tuple(sorted((county,county2))))
                  
    for pairs in poss_monos:
        mrca1 = mrcas[pairs[0]]
        mrca2 = mrcas[pairs[1]]
        
        if float(mrca1.annotations['height'].value) > float(mrca2.annotations['height'].value):
            if mrca2 not in paths[mrca1]:
                monos.add(pairs[0])
                monos.add(pairs[1])
            else:
                monos.add(pairs[1])
                spillover[pairs[0]] = pairs[1]
        else:
            if mrca1 not in paths[mrca2]:
                monos.add(pairs[1])
                monos.add(pairs[0])
            else:
                monos.add(pairs[0])
                spillover[pairs[1]] = pairs[0]
                
    
    multi_counties[nde]["polys"] = polyphyletic
    multi_counties[nde]["monos"] = monos
    multi_counties[nde]["spillover"] = spillover

In [158]:
for k,v in multi_counties.items():
    print(v['polys'])

{}
{}
{}
{}
{'Middlesex': 'New_Haven', 'New_Haven': 'Middlesex'}
{}
{}
{}
{}
{}
{'Oswego': 'Madison', 'Onondaga': 'Madison', 'Madison': 'Onondaga'}
{}
{}
{}
{}
{'Bristol': 'Plymouth', 'Plymouth': 'Bristol'}


In [159]:
for k,v in multi_counties.items():
    print(v['spillover'])

{}
{'Plymouth': 'Bristol'}
{'Bristol': 'Plymouth'}
{'Onondaga': 'Oswego', 'Madison': 'Onondaga', 'Oneida': 'Oswego'}
{}
{'Onondaga': 'Madison'}
{}
{}
{}
{'Plymouth': 'Barnstable', 'Barnstable': 'Bristol'}
{'Onondaga': 'Oneida', 'Oswego': 'Oneida', 'Madison': 'Oneida'}
{}
{}
{}
{}
{'Plymouth': 'Norfolk', 'Bristol': 'Norfolk', 'Essex': 'Middlesex'}


In [161]:
for k,v in multi_counties.items():
    print(v['monos'])
    print(len(clusters[k]))

{'Suffolk', 'Onondaga'}
4
{'Bristol'}
6
{'Worcester', 'Middlesex', 'Bristol', 'Barnstable', 'Plymouth', 'Norfolk'}
22
{'Oswego', 'Madison', 'Oneida', 'Chemung', 'Onondaga'}
48
set()
6
{'Oswego', 'Madison', 'Oneida', 'Wayne', 'Seneca', 'Onondaga'}
16
{'Windham', 'New_London', 'Fairfield', 'New_Haven'}
5
{'Essex', 'Worcester'}
3
{'New_London', 'Madison', 'Onondaga', 'St_Lawrence'}
5
{'Bristol', 'Barnstable'}
6
{'Montgomery', 'Oswego', 'Madison', 'Oneida', 'St._Lawrence', 'Onondaga'}
65
{'Orange', 'Sullivan', 'Wayne'}
4
{'Oswego', 'Onondaga', 'Steuben'}
9
{'New_London', 'Ostego'}
10
{'Oswego', 'Onondaga'}
2
{'Middlesex', 'Essex', 'Norfolk'}
32
